# Домашнее задание по теме «Рекомендации на основе содержания»

Задание

Вопросы по заданию

Преподаватель: Наталья Баданина, Юлия Пономарева, Егор Шишковец

1. Использовать датасет MovieLens.
2. Построить рекомендации (регрессия, предсказываем оценку) на фичах:

*  TF-IDF на тегах и жанрах;
*  средние оценки (+ median, variance и т. д.) пользователя и фильма.

3. Оценить RMSE на тестовой выборке.



## 1. Использовать датасет MovieLens.

In [292]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics.pairwise import cosine_distances
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [254]:
links = pd.read_csv('ml-latest-small/links.csv')
movies = pd.read_csv('ml-latest-small/movies.csv')
ratings = pd.read_csv('ml-latest-small/ratings.csv')
tags = pd.read_csv('ml-latest-small/tags.csv')

In [255]:
print('links')
print(links.head(1))

print('movies')
print(movies.head(1))

print('ratings')
print(ratings.head(1))

print('tags')
print(tags.head(1))

links
   movieId  imdbId  tmdbId
0        1  114709   862.0
movies
   movieId             title                                       genres
0        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy
ratings
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
tags
   userId  movieId    tag   timestamp
0       2    60756  funny  1445714994


## 2. Построить рекомендации (регрессия, предсказываем оценку) на фичах:

*  TF-IDF на тегах и жанрах;
*  средние оценки (+ median, variance и т. д.) пользователя и фильма.

In [256]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [257]:
movies_with_ratings = movies.merge(ratings, on='movieId')
movies_with_ratings.drop(['userId', 'timestamp'], axis= 1 , inplace= True )
movies_with_ratings

,movieId,title,genres,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.0
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.0
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.5
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2.5
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.5
...,...,...,...,...
100831,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,4.0
100832,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,3.5
100833,193585,Flint (2017),Drama,3.5
100834,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,3.5


In [258]:
movies_with_tags = movies_with_ratings.merge(tags, on='movieId')
movies_with_tags.head()

,movieId,title,genres,rating,userId,tag,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.0,336,pixar,1139045764
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.0,474,pixar,1137206825
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.0,567,fun,1525286013
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.0,336,pixar,1139045764
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.0,474,pixar,1137206825


In [259]:
unique_movies = movies_with_tags[['movieId', 'title', 'genres', 'rating']].drop_duplicates(subset=['movieId', 'title', 'genres'])

unique_movies = unique_movies.reset_index()
unique_movies.shape

(1554, 5)

In [260]:
unique_movies

,index,movieId,title,genres,rating
0,0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.0
1,645,2,Jumanji (1995),Adventure|Children|Fantasy,4.0
2,1085,3,Grumpier Old Men (1995),Comedy|Romance,4.0
3,1189,5,Father of the Bride Part II (1995),Comedy,5.0
4,1287,7,Sabrina (1995),Comedy|Romance,4.0
...,...,...,...,...,...
1549,233148,183611,Game Night (2018),Action|Comedy|Crime|Horror,4.0
1550,233151,184471,Tomb Raider (2018),Action|Adventure|Fantasy,3.5
1551,233163,187593,Deadpool 2 (2018),Action|Comedy|Sci-Fi,5.0
1552,233199,187595,Solo: A Star Wars Story (2018),Action|Adventure|Children|Sci-Fi,4.0


Для удобства отмасштабируем идентификаторы фильмов таким образом, чтобы они начинались с 0 и заканчивались на n_movies-1

In [261]:
movies_values = unique_movies['movieId'].unique()

unique_movies['movieId'] = unique_movies['movieId'].apply(lambda f: np.where(movies_values == f)[0][0])

In [262]:
unique_movies.head()

,index,movieId,title,genres,rating
0,0,0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.0
1,645,1,Jumanji (1995),Adventure|Children|Fantasy,4.0
2,1085,2,Grumpier Old Men (1995),Comedy|Romance,4.0
3,1189,3,Father of the Bride Part II (1995),Comedy,5.0
4,1287,4,Sabrina (1995),Comedy|Romance,4.0


In [263]:
unique_movies['genres'] = unique_movies['genres'].apply(lambda x: ' '.join(x.split('|')))
unique_movies['text_feature'] = unique_movies.apply(lambda x: x['title'].lower() + ' ' + x['genres'].lower(), axis = 1)
unique_movies = unique_movies.sort_values('movieId')

unique_movies[:5]

,index,movieId,title,genres,rating,text_feature
0,0,0,Toy Story (1995),Adventure Animation Children Comedy Fantasy,4.0,toy story (1995) adventure animation children ...
1,645,1,Jumanji (1995),Adventure Children Fantasy,4.0,jumanji (1995) adventure children fantasy
2,1085,2,Grumpier Old Men (1995),Comedy Romance,4.0,grumpier old men (1995) comedy romance
3,1189,3,Father of the Bride Part II (1995),Comedy,5.0,father of the bride part ii (1995) comedy
4,1287,4,Sabrina (1995),Comedy Romance,4.0,sabrina (1995) comedy romance


In [264]:
vectorizer = TfidfVectorizer()
tfidf_features = vectorizer.fit_transform(unique_movies['text_feature']).toarray()

tfidf_features.shape

(1554, 2485)

In [265]:
pd.DataFrame(tfidf_features)

,0,1,2,3,4,5,6,7,8,9,...,2475,2476,2477,2478,2479,2480,2481,2482,2483,2484
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1549,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1550,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1551,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1552,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [266]:
[u for u, v in vectorizer.vocabulary_.items() if v == 4]

['101']

Вычислим расстояния

In [267]:
cosine_similarity = cosine_distances(tfidf_features)

In [268]:
cosine_similarity.shape

(1554, 1554)

Зная попарные косинусные расстояния, найдем топ 5 кандидатов для каждого фильма

In [269]:
top=5
movies_sim = []
for i in range(len(cosine_similarity)):
    neighbors = (cosine_similarity[i]).argsort()[1:top+1] # считаем от 1 до top1 + 1, 
    # так как ближайшее расстояние у вектора до самого себя
    movies_sim.append(neighbors)
movies_sim = np.array(movies_sim)

In [270]:
movies_sim.shape

(1554, 5)

In [271]:
unique_movies

,index,movieId,title,genres,rating,text_feature
0,0,0,Toy Story (1995),Adventure Animation Children Comedy Fantasy,4.0,toy story (1995) adventure animation children ...
1,645,1,Jumanji (1995),Adventure Children Fantasy,4.0,jumanji (1995) adventure children fantasy
2,1085,2,Grumpier Old Men (1995),Comedy Romance,4.0,grumpier old men (1995) comedy romance
3,1189,3,Father of the Bride Part II (1995),Comedy,5.0,father of the bride part ii (1995) comedy
4,1287,4,Sabrina (1995),Comedy Romance,4.0,sabrina (1995) comedy romance
...,...,...,...,...,...,...
1549,233148,1549,Game Night (2018),Action Comedy Crime Horror,4.0,game night (2018) action comedy crime horror
1550,233151,1550,Tomb Raider (2018),Action Adventure Fantasy,3.5,tomb raider (2018) action adventure fantasy
1551,233163,1551,Deadpool 2 (2018),Action Comedy Sci-Fi,5.0,deadpool 2 (2018) action comedy sci-fi
1552,233199,1552,Solo: A Star Wars Story (2018),Action Adventure Children Sci-Fi,4.0,solo: a star wars story (2018) action adventur...


Составим датафрейм для того, чтобы понять по описаниям, какие фильмы похожи на какие

In [272]:
movie_id_name = dict(unique_movies.set_index('movieId')['text_feature'])
movies_top_df = pd.DataFrame(movies_sim, columns = ['top1', 'top2', 'top3', 'top4', 'top5'])
movies_top_df['top1'] = movies_top_df['top1'].apply(lambda x: movie_id_name[x])
movies_top_df['top2'] = movies_top_df['top2'].apply(lambda x: movie_id_name[x])
movies_top_df['top3'] = movies_top_df['top3'].apply(lambda x: movie_id_name[x])
movies_top_df['top4'] = movies_top_df['top4'].apply(lambda x: movie_id_name[x])
movies_top_df['top5'] = movies_top_df['top5'].apply(lambda x: movie_id_name[x])

movies_top_df['movieId'] = movies_top_df.index

In [273]:
movies_recs = (
    unique_movies.merge(
        movies_top_df,
        left_on='movieId', right_on='movieId'
    )
)

In [274]:
movies_recs[:5]

,index,movieId,title,genres,rating,text_feature,top1,top2,top3,top4,top5
0,0,0,Toy Story (1995),Adventure Animation Children Comedy Fantasy,4.0,toy story (1995) adventure animation children ...,toy story 2 (1999) adventure animation childre...,jumanji (1995) adventure children fantasy,shrek (2001) adventure animation children come...,"christmas story, a (1983) children comedy",l.a. story (1991) comedy romance
1,645,1,Jumanji (1995),Adventure Children Fantasy,4.0,jumanji (1995) adventure children fantasy,jumanji: welcome to the jungle (2017) action a...,toy story (1995) adventure animation children ...,babe (1995) children drama,shrek (2001) adventure animation children come...,"little princess, a (1995) children drama"
2,1085,2,Grumpier Old Men (1995),Comedy Romance,4.0,grumpier old men (1995) comedy romance,old school (2003) comedy,old boy (2003) mystery thriller,mystery men (1999) action comedy fantasy,"40-year-old virgin, the (2005) comedy romance",x-men (2000) action adventure sci-fi
3,1189,3,Father of the Bride Part II (1995),Comedy,5.0,father of the bride part ii (1995) comedy,father of the bride (1991) comedy,father of the bride (1950) comedy,"godfather: part ii, the (1974) crime drama",in the name of the father (1993) drama,"karate kid, part ii, the (1986) action adventu..."
4,1287,4,Sabrina (1995),Comedy Romance,4.0,sabrina (1995) comedy romance,sabrina (1954) comedy romance,clueless (1995) comedy romance,persuasion (1995) drama romance,"american president, the (1995) comedy drama ro...",mighty aphrodite (1995) comedy drama romance


In [275]:
movies_recs[movies_recs['title'].apply(lambda x: 'Star Wars' in x)]

,index,movieId,title,genres,rating,text_feature,top1,top2,top3,top4,top5
65,12542,65,Star Wars: Episode IV - A New Hope (1977),Action Adventure Sci-Fi,5.0,star wars: episode iv - a new hope (1977) acti...,star wars: episode i - the phantom menace (199...,star wars: episode iii - revenge of the sith (...,star wars: episode vi - return of the jedi (19...,star wars: episode v - the empire strikes back...,solo: a star wars story (2018) action adventur...
287,112879,287,Star Wars: Episode V - The Empire Strikes Back...,Action Adventure Sci-Fi,5.0,star wars: episode v - the empire strikes back...,star wars: episode i - the phantom menace (199...,star wars: episode iii - revenge of the sith (...,star wars: episode vi - return of the jedi (19...,star wars: episode iv - a new hope (1977) acti...,solo: a star wars story (2018) action adventur...
298,118998,298,Star Wars: Episode VI - Return of the Jedi (1983),Action Adventure Sci-Fi,5.0,star wars: episode vi - return of the jedi (19...,star wars: episode iii - revenge of the sith (...,star wars: episode i - the phantom menace (199...,star wars: episode v - the empire strikes back...,star wars: episode iv - a new hope (1977) acti...,solo: a star wars story (2018) action adventur...
586,143748,586,Star Wars: Episode I - The Phantom Menace (1999),Action Adventure Sci-Fi,4.0,star wars: episode i - the phantom menace (199...,star wars: episode iii - revenge of the sith (...,star wars: episode vi - return of the jedi (19...,star wars: episode v - the empire strikes back...,star wars: episode iv - a new hope (1977) acti...,solo: a star wars story (2018) action adventur...
1299,201701,1299,Star Wars: Episode III - Revenge of the Sith (...,Action Adventure Sci-Fi,1.0,star wars: episode iii - revenge of the sith (...,star wars: episode i - the phantom menace (199...,star wars: episode vi - return of the jedi (19...,star wars: episode v - the empire strikes back...,star wars: episode iv - a new hope (1977) acti...,solo: a star wars story (2018) action adventur...
1552,233199,1552,Solo: A Star Wars Story (2018),Action Adventure Children Sci-Fi,4.0,solo: a star wars story (2018) action adventur...,star wars: episode i - the phantom menace (199...,deadpool 2 (2018) action comedy sci-fi,star wars: episode iii - revenge of the sith (...,star wars: episode vi - return of the jedi (19...,star wars: episode iv - a new hope (1977) acti...


Перейдем к предсказанию рейтинга

In [297]:
train_data, test_data = train_test_split(ratings, test_size=0.01)

print('Train shape: {}'.format(train_data.shape))
print('Test shape: {}'.format(test_data.shape))

Train shape: (99827, 4)
Test shape: (1009, 4)


Посчитаем метрику на тесте, если бы мы предсказали всем средний рейтинг

In [298]:
train_data['rating'].mean()

3.501292235567532

In [299]:
const_default = 3.5
test_data['constant_predict'] = const_default
print(f"RMSE metric: {np.sqrt(mean_squared_error(test_data['constant_predict'], test_data['rating']))}")


RMSE metric: 1.0412693328758467
